Projeto

In [ ]:
#!pip install SoccerNet --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.2/85.2 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.3/104.3 MB 8.9 MB/s eta 0:00:00


In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import SoccerNet
from SoccerNet.Downloader import SoccerNetDownloader

password = "s0cc3rn3t"

def unlabeled_dataset(LocalDirectory):
    myDownloader = SoccerNetDownloader(LocalDirectory=LocalDirectory)
    myDownloader.password = password
    myDownloader.downloadGames(files=["1_224p.mkv", "2_224p.mkv"], split=["train","valid","test","challenge"])

def labeled_dataset(LocalDirectory):
    myDownloader = SoccerNetDownloader(LocalDirectory="Football-Dataset/")
    myDownloader.password = password
    myDownloader.downloadGames(files=["Labels-v2.json"], split=["train","valid","test"])

unlabeled_dataset("Football-Dataset/")
#labeled_dataset("Football-Dataset/")

Football-Dataset/england_epl\2014-2015\2015-02-21 - 18-00 Chelsea 1 - 1 Burnley\1_224p.mkv already exists
Football-Dataset/england_epl\2014-2015\2015-02-21 - 18-00 Chelsea 1 - 1 Burnley\2_224p.mkv already exists


KeyboardInterrupt: 

In [ ]:
#!pip install transformers
#!pip install ffmpeg
#!pip install torch
#!pip install ffmpeg-python
#!pip install accelerate

  Preparing metadata (setup.py) ... done
  Created wheel for ffmpeg: filename=ffmpeg-1.4-py3-none-any.whl size=6082 sha256=da0ade5417ae8a280a82929758fa9a867283d7d9b89d3a7c89768315a3f25099
  Stored in directory: /root/.cache/pip/wheels/8e/7a/69/cd6aeb83b126a7f04cbe7c9d929028dc52a6e7d525ff56003a
Successfully built ffmpeg
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.

In [2]:
import ffmpeg
from pprint import pprint
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
import torch
import os
import shutil
import time

In [11]:

def build_model(model_id):
    device = "cuda:0" if torch.cuda.is_available() else "cpu"
    torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32


    model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, use_safetensors=True
                                                      )
    model.to(device)

    processor = AutoProcessor.from_pretrained(model_id, language="en")

    pipe = pipeline(
        "automatic-speech-recognition",
        model=model,
        tokenizer=processor.tokenizer,
        feature_extractor=processor.feature_extractor,
        max_new_tokens=128,
        chunk_length_s=30,
        batch_size=16,
        return_timestamps=True,
        torch_dtype=torch_dtype,
        device=device,
    )

    return pipe

#model_id = "openai/whisper-base" #"openai/whisper-large-v3"
model_ids = ["openai/whisper-base", "openai/whisper-small", "openai/whisper-medium", "openai/whisper-large"]
#pipe = build_model(model_id="openai/whisper-base")

In [7]:
# environment variables
#local_dir = "drive/MyDrive/DTX/Football-Dataset"
dir = "Football-Dataset/england_epl/"

def update_txt_file(text, path):
    with open(path, "w", encoding="utf-8") as f:
        f.write(text)


def audio_to_text(pipe, model_id):
  for year in os.listdir(dir):
      games = os.path.join(dir, year)
      for game in os.listdir(games):
          if game != ".ipynb_checkpoints":
              game_dir = os.path.join(games, game)
              frag1_dir = os.path.join(game_dir, "fragments_1_224p")
              frag2_dir = os.path.join(game_dir, "fragments_2_224p")

              for audioFrag1 in os.listdir(frag1_dir):
                result = pipe(frag1_dir + '/' + audioFrag1, return_timestamps=True,  generate_kwargs = {"language": "english", "task": "translate"})
                update_txt_file(result['text'], frag1_dir + f'/{audioFrag1[:-4]}__{model_id[7:]}.txt')

              for audioFrag2 in os.listdir(frag2_dir):
                result = pipe(frag2_dir + '/' + audioFrag2, return_timestamps=True,  generate_kwargs = {"language": "english", "task": "translate"})
                update_txt_file(result['text'], frag2_dir + f'/{audioFrag2[:-4]}__{model_id[7:]}.txt')


def clear_nonvideos(path):
    for t in os.listdir(path):
        if os.path.isdir(path + '/' + t):
          shutil.rmtree(path + '/' + t)
        else:
          if not t.endswith(".mkv"):
             os.remove(path + '/' + t)


def fragment_dataset_audio(audio_cut_length, offset):
  for year in os.listdir(dir):
      games = os.path.join(dir, year)
      for game in os.listdir(games):
          game_dir = os.path.join(games, game)

          if game != ".ipynb_checkpoints":
            clear_nonvideos(game_dir)
            for videoFile in os.listdir(game_dir):
                pathVideo = game_dir + "/" + videoFile
                videoFragments = game_dir + f"/fragments_{videoFile[:-4]}"
                if not os.path.exists(videoFragments):
                  os.mkdir(videoFragments)

                duration = float(ffmpeg.probe(pathVideo)["format"]["duration"])
                steps = duration/audio_cut_length
                for n in range(int(steps)+1):
                  ffmpeg.input(pathVideo, ss=(n*audio_cut_length)-offset, to=(n+1)*audio_cut_length).output(videoFragments + f"/fragment_{n}.wav").run(quiet=True, overwrite_output=True)


#fragment_dataset_audio(audio_cut_length=30, offset=2)
#audio_to_text()

In [13]:
# Benchmark locally and globally
def benchmark():
  #for m in model_ids:
    m = "openai/whisper-medium"
    fragment_dataset_audio(audio_cut_length=30, offset=2)
    start_time = time.time()
    pipe = build_model(model_id=m)
    audio_to_text(pipe, m)
    print(f"iter {m}")
    print(m, time.time() - start_time)

benchmark()

KeyboardInterrupt: 

In [ ]:
# Iterate through folders to apply zero-shot classification
classifier = pipeline(
    "zero-shot-classification",
    model="facebook/bart-large-mnli",
)

#sequence_to_classify = result['text']
sequence_to_classify = " Mark Mitch For the continued work the dry Sun A half minute in and Continue produces work from the top draw to put Liverpool ahead Wow, what a fantastic strike that is Stunning goal Absolutely stunning goal just looking at his technique the goal. I'm just looking at his technique. I mean, it's a umbrella to be slow build up from New York for nobody in the ground for a minute was going to hit him. And there he bends it with power."
candidate_labels = ['danger moment', 'interesting', 'uninteresting', 'moment without danger']
res = classifier(sequence_to_classify, candidate_labels)
print(res)

# Benchmark the different versions of the model
model_versions = ["facebook/bart-large-mnli", "???"]


In [ ]:
# New Dataset Evaluation - Simão e Tiago(trabalha este?)
def build_labeled_dataset():
    base_path = "Football-Dataset/england_epl/"


    for year in os.listdir(base_path):
        games = os.path.join(base_path, year)
        for game in os.listdir(games):
            fragments_1 = os.path.join(games, game, "fragments_1_224p")
            fragments_2 = os.path.join(games, game, "fragments_2_224p")

            write_fragments(folder=fragments_1, league="england_epl", game=game)
            #write_fragments(folder=fragments_2, league="england_epl", game=game)

def write_fragments(folder, league, game, audio_cut_length=30, offset=2):

    num_transcriptions = len(os.listdir(folder))//2
    # ex: (fragment_20.txt, 20)  -> (fragment file name, fragment_number)
    fragment_file_names = [("fragment_" + str(i) + f"__{model_id}.txt", i) for i in range(num_transcriptions)]


    for fragment_file in fragment_file_names:
        fragment_path = os.path.join(folder, fragment_file[0])
        # These values are counted in seconds
        start_time = fragment_file[1] * audio_cut_length - offset
        end_time = (fragment_file[1]+1) * audio_cut_length

        with open(fragment_path, "r") as f:
          pass


build_labeled_dataset()



IndentationError: expected an indented block after 'with' statement on line 28 (<ipython-input-25-9812d90212e9>, line 32)

In [ ]:
id;league;game;fragment_start_time;fragment_end_time;label;